# Data import

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud.bigquery import Dataset

client = bigquery.Client()
     

In [2]:
QUERY = (
    """SELECT unique_client_id, 
    visitNumber, 
    channelgrouping, 
    device, 
    date, 
    transaction, 
    revenue    
     FROM `gap-bq-api-2016.attribution_modeling.11_final_view_lite` 
     where unique_client_id in (select unique_client_id from `gap-bq-api-2016.attribution_modeling.11_final_view_lite` 
     where visitNumber = 1)
   limit 1000  """)


In [3]:
projectid = "gap-bq-api-2016"
df = pd.read_gbq(QUERY, projectid, dialect='standard')

Requesting query... ok.
Job ID: job_6_sjBzvHj9KAjF9LEyGKdDEbKhfI
Query running...
Query done.
Processed: 169.9 MB
Standard price: $0.00 USD

Retrieving results...
  Got page: 1; 100% done. Elapsed 9.82 s.
Got 1000 rows.

Total time taken 9.84 s.
Finished at 2017-12-13 22:01:25.


In [4]:
df.head()

,unique_client_id,visitNumber,channelgrouping,device,date,transaction,revenue
0,1048375319162654318,1,Email,desktop,20171101,None,NaN
1,6471227363831698677,1,Email,desktop,20171101,None,NaN
2,8411127112042974805,1,Email,mobile,20171101,None,NaN
3,6398481703148091424,1,Email,mobile,20171101,None,NaN
4,5676688380058563631,1,Email,mobile,20171101,None,NaN


In [18]:
df.date=df.date.astype(int)
df['max_dt_dataset'] = df['date'].max()
df = df.fillna(value = 0)    
df.dtypes
df.head()

,unique_client_id,visitNumber,channelgrouping,device,date,transaction,revenue,cnt_key,max_dt_dataset
855,1004756623444113461,2,Local Listing,mobile,20171126,0,0.0,1,20171130
580,1008431904268064112,1,Display,tablet,20171119,0,0.0,1,20171130
662,1008610531958870471,1,Display,tablet,20171130,0,0.0,1,20171130
6,1021847865975762180,1,Email,desktop,20171101,0,0.0,1,20171130
23,1032029549807475297,2,Email,desktop,20171102,0,0.0,1,20171130


In [16]:
df = df.sort_values(by=['unique_client_id', 'visitNumber'], ascending=True)
df.head()

,unique_client_id,visitNumber,channelgrouping,device,date,transaction,revenue,cnt_key,max_dt_dataset
855,1004756623444113461,2,Local Listing,mobile,20171126,0,0.0,1,0
580,1008431904268064112,1,Display,tablet,20171119,0,0.0,1,0
662,1008610531958870471,1,Display,tablet,20171130,0,0.0,1,0
6,1021847865975762180,1,Email,desktop,20171101,0,0.0,1,0
23,1032029549807475297,2,Email,desktop,20171102,0,0.0,1,0


# Markov Chains [R integration to validate]

In [ ]:
#Markov chains
QUERYMKV = (
    """SELECT unique_client_id, 
    visitNumber, 
    channelgrouping, 
    device, 
    date, 
    transaction, 
    revenue    
     FROM `gap-bq-api-2016.attribution_modeling.markov_view` 
     where unique_client_id in (select unique_client_id from `gap-bq-api-2016.attribution_modeling.13_step` 
     where visitNumber = 1)
     """)

projectid = "gap-bq-api-2016"
mkv_df = pd.read_gbq(QUERYMKV, projectid, dialect='standard')
idx_client = mkv_df['unique_client_id'].drop_duplicates() 
df['cnt_key'] = mkv_df.groupby(['unique_client_id'])['unique_client_id'].transform('count')
mkv_df = pd.DataFrame(columns = ['unique_client_id', 'path', 'device', 'revenue', 'revInd'], index=idx_client)
chn_grp = ''
dev_grp = ''
ant_kid = ''
idx = 0
for index, row in df.iterrows():
    if idx == row['cnt_key'] and idx == 1:
            mkv_df = mkv_df.append({'unique_client_id' : row['unique_client_id'],
            'path' : row['channelgrouping'], 
            'device' : row['device'], 
            'revenue' : row['revenue'],                        
            'revInd' : row['revenue_ind']}, ignore_index=True)
            idx = idx-1
    elif idx > 1:
        if row['revenue_ind'] == 0:
            if chn_grp == '' and dev_grp == '':
                dev_grp = row['device']
                chn_grp = row['channelgrouping']
                idx = idx-1
            elif chn_grp == '':
                chn_grp = row['channelgrouping']
                idx = idx-1
            elif dev_grp == '':
                dev_grp = row['device']
                idx = idx-1
            elif chn_grp != '' and dev_grp != '':
                chn_grp = chn_grp + ' > ' + row['channelgrouping']
                dev_grp = dev_grp + ' > ' + row['device']
                idx = idx-1
            
        elif row['revenue_ind'] == 1:
            chn_grp = chn_grp + ' > ' + row['channelgrouping']
            dev_grp = dev_grp + ' > ' + row['device']
            mkv_df = mkv_df.append({'unique_client_id' : row['unique_client_id'],
            'path' : chn_grp, 
            'device' : dev_grp, 
            'revenue' : row['revenue'],   
            'revInd' : row['revenue_ind']}, ignore_index=True)
            chn_grp = ''
            dev_grp = ''
            idx = idx-1    
    elif idx == 1:
        mkv_df = mkv_df.append({'unique_client_id' : row['unique_client_id'],
        'path' : chn_grp, 
        'device' : dev_grp, 
        'revInd' : row['revenue_ind']}, ignore_index=True)
        chn_grp = ''
        dev_grp = ''
        idx = idx-1
    elif idx == 0:
        idx = row['cnt_key']
        ant_kid = row['unique_client_id']
    
        if row['revenue_ind'] == 0:
            if chn_grp == '' and dev_grp == '':
                dev_grp = row['device']
                chn_grp = row['channelgrouping']
                idx = idx-1
            elif chn_grp == '':
                chn_grp = row['channelgrouping']
                idx = idx-1
            elif dev_grp == '':
                dev_grp = row['device']
                idx = idx-1
            elif chn_grp != '' and dev_grp != '':
                chn_grp = chn_grp + ' > ' + row['channelgrouping']
                dev_grp = dev_grp + ' > ' + row['device']
                idx = idx-1
            
        elif row['revenue_ind'] == 1:
            chn_grp = str(chn_grp + ' > ' + row['channelgrouping'])
            dev_grp = dev_grp + ' > ' + row['device']
            mkv_df = mkv_df.append({'unique_client_id' : row['unique_client_id'],
            'path' : chn_grp, 
            'device' : dev_grp, 
            'revenue' : row['revenue'],   
            'revInd' : row['revenue_ind']}, ignore_index=True)
            chn_grp = ''
            dev_grp = ''
            idx = idx-1
mkv_df['path'] = mkv_df['path'].str.replace("Direct", "").str.replace("Others", "").str.replace(">  >", ">").str.replace('> ', "")
mkv_df['device'] = mkv_df['device'].str.replace('> ', "")

mkv_df = mkv_df.drop_duplicates()
mkv_df = mkv_df.fillna(0)

mkv_df[:1000]

Requesting query... 